### tflite custom operator

- https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/custom_operators.md

In [1]:
import tensorflow as tf
import numpy as np
import os

#### Model

- y = arcsin(x + 0.2)

In [2]:
offset_initializer = tf.constant_initializer(value=0.0)

In [3]:
offset = tf.get_variable("offset", [1,], tf.float32, initializer=offset_initializer)
x = tf.placeholder(tf.float32, shape=(1,17))
y = tf.asin(x + offset)
y_ = tf.placeholder(tf.float32, shape=(1,17))
loss = tf.reduce_sum(tf.square(y - y_))
optimizer = tf.train.GradientDescentOptimizer(0.001)
train = optimizer.minimize(loss)

#### Input data

In [4]:
np_x = np.array([-0.9, -0.8, -0.7, -0.6, -0.5, -0.4, -0.3, -0.2, -0.1, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7], dtype=np.float32, ndmin=2)
np_y_ = np.array([-0.7754, -0.6435, -0.5236, -0.4115, -0.3047, -0.2014, -0.1002, 0.0000, 0.1002, 0.2014, 0.3047, 0.4115, 0.5236,
                  0.6435, 0.7754, 0.9273, 1.1198], dtype=np.float32, ndmin=2)

#### Fit & convert to tflite model

- https://www.tensorflow.org/lite/convert/python_api

In [5]:
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(500):
        sess.run(train, feed_dict={x: np_x, y_: np_y_})
    
    print('\t loss =   %E' % sess.run(loss, feed_dict={x: np_x, y_: np_y_}))
    print('\t offset = %.4f' % sess.run(offset, feed_dict={x: np_x, y_: np_y_}))
    
    converter = tf.contrib.lite.TFLiteConverter.from_session(sess, [x], [y])
    converter.allow_custom_ops=True
    tflite_model = converter.convert()
    open("tflite_op_sine.tflite", "wb").write(tflite_model)

	 loss =   7.084212E-09
	 offset = 0.2000
INFO:tensorflow:Froze 1 variables.
INFO:tensorflow:Converted 1 variables to const ops.
